<a href="https://colab.research.google.com/github/skrzypczykt/MAchineLearningProjects/blob/main/NeuralNetworksTutorials/keras/CNN%20%2B%20generatory/Sieci_08_conv_nets_02_cats_vs_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sieci splotowe

Sieci neuronowe są bardzo często wykorzystywane do analizy obrazu albo dźwięku. Nie wykorzystuje się wtedy zwykłych sieci sekwencyjnych o połączeniach każdy z każdym:
* do analizy obrazu używa się splotowych (*konwolucyjnych*) sieci neuronowych (*convolutional neural networks*, **CNN**)
* do analizy dźwięku używa się często splotowych sieci rekurencyjnych (*recurrent neural networks*, **RNN**)

My jednak zrobimy coś nieco ciekawszego/zabawniejszego, pobawimy się w rozpoznawanie czy na zdjęciu jest pies czy kot.  

Link do danych dostaliście mailem. 

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

Wczytujemy nazwy plików graficznych. Jako `cat_dir` oraz `dog_dir` wprowadź ścieżkę do folderu treningowego `psy_vs_koty` odpowiednio z kotami i psami.

In [ ]:
cat_dir = r'_____'
dog_dir = r'_____'
cat_images = os.listdir(_____)
dog_images = os.listdir(_____)

# wyświetlamy podsumowanie
print('Obrazy psów:', ', '.join(dog_images[:3]), '... razem {}'.format(len(dog_images)))
print('Obrazy kotów:', ', '.join(cat_images[:3]), '... razem {}'.format(len(cat_images)))

Tworzymy funkcję do wyświetlania zdjęć (tutaj nic nie musicie zmieniać, ale przejrzyjcie kod aby mieć ogólne zrozumienie co robi):

In [ ]:
import os.path as op
from keras.preprocessing.image import load_img

def show_image(image_type, image_index, target_size=None):
    # ogarniamy ścieżki
    img_dirs = {'cat': cat_dir, 'dog': dog_dir}
    fname = '.'.join([image_type, str(image_index), 'jpg'])
    full_img_path = op.join(img_dirs[image_type], fname)

    # wczytujemy plik
    img = load_img(full_img_path, target_size=target_size)
    
    # wyświetlamy
    plt.imshow(img)
    plt.title(fname)
    plt.axis('off')

Wyświetlamy wybrane zdjęcie podając do funkcji rodzaj zdjęcia (`'cat'` albo `'dog'`) oraz jego identyfikator numeryczny (np. `23`). Spróbuj poniżej wyświetlić zdjęcie `cat.4586.jpg`:

In [ ]:
image_type = ____
image_index = ____
show_image(image_type, image_index)

Zmień kod poniżej aby wczytać inne zdjęcie. Zrób tak kilka razy aby przejrzeć różne zdjęcia.
* Przejrzyj co najmniej 3 zdjęcia kotów i 3 zdjęcia psów - jak porównałabyś/porównałbyś zróżnicowanie i złożoność obrazów psów i kotów w porównaniu do poprzedniego datasetu, Fashion MNIST?
* Do datasetu, z którego korzystamy zgłaszane były problemy. Sprawdź pliki omawiane w komentarzach poniżej:
  > Image cat.7377.jpg is labelled as a cat. However it is a person. Not a dog, not a cat. Why is it in the data?
  
  oraz:
  
  > Unless there is a cat in 1 pixel of this image I can't see, image cat.4085.jpg is clearly a dog to me.
  
* Sprawdź jeszcze co najmniej 3 z poniższych zdjęć:
  - cat.4338.jpg
  - dog.1043.jpg
  - dog.1773.jpg
  - cat.3216.jpg
  - cat.4688.jpg
  - cat.5351.jpg
  - dog.8898.jpg
  - dog.10237.jpg
  - cat.11184.jpg  
  
  Jak myślisz dlaczego te zdjęcia znalazły się w zbiorze danych treningowych?
  
Nie będziemy w ramach zajęć poprawiać tego datasetu, zresztą warto mieć świadomość, że wiele z publicznie dostępnych zbiorów danych nie jest perfekcyjna i może zawierać błędy. Stosowane przez nas algorytmy powinny być odporne na obecność takich drobnych błędów.

In [ ]:
image_type = ____
image_index = ____
show_image(_______, _______)

Wiele zdjęć jest w wysokiej rozdzielczości:

In [ ]:
image_type = 'dog'
image_index = 12051
show_image(image_type, image_index)

Przy czym poszczególne zdjęcia są w różnych rozmiarach, musimy je przekształcić wszystkie na ten sam rozmiar. Aby oszczędzić czasu na treningu zredukujemy zdjęcia do rozmiaru 80 na 80 pikseli. Korzystamy z argumentu nazwowego `target_size` w napisanej przez nas funkcji `show_image` aby wyświetlić pomniejszony obraz (chcemy 80 na 80 pikseli):

In [ ]:
image_type = 'dog'
image_index = 12051
show_image(image_type, image_index, ____________=(____, ____))

Na takich obrazach będzie uczyć się sieć, cały czas da się rozróżnić w obrazie psa, ale detali jest mniej.

## `ImageDataGenerator`
Ponieważ będziemy teraz korzystać z większych obrazów niż w przypadku danych Fashion MNIST (`80 x 80` w przeciwieństwie do wcześniejszych `28 x 28`), a ponadto sami musielibyśmy je wszystkie wczytywać, przeskalowywać do odpowiedniego rozmiaru i budować macierz - zastosujemy inne, bardziej elastyczne rozwiązanie.  
Zamiast wczytywać wszystkie obrazy i przetrzymywać je na raz w pamięci komputera jako `X_train` oraz `X_test` będziemy obrazy wczytywać dynamicznie podczas treningu z dysku. Takie rozwiązanie ma dodatkowe zalety przy *image augmentation*, z którego skorzystamy później. Korzystamy do tego z obiektu `ImageDataGenerator`

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

Tworząc obiekt `ImageDataGenerator` skorzystamy z argumentu nazwowego `rescale` pozwalającego na przeskalowanie wartości pikseli obrazu. Aby przeskalować wartości do zakresu 0 - 1 podamy temu argumentowi wartość `1 / 255` - spowoduje to, że każdy piksel zostanie pomnożony przez tę wartość, a więc podzielony przez `255` - maksymalną wartość piksela w danych RGB.

In [ ]:
scaling = _________
data_gen = ImageDataGenerator(rescale=scaling)

Teraz skorzystamy z metody `.flow_from_directory` utworzonego obiektu aby stworzyć wersję generatora, która pobiera obrazy z folderu. Jako pierwszy argument podajemy ścieżkę do folderu `train` - generator sam już rozpozna, że mamy do czynienia z dwoma klasami: `cat` oraz `dog` na podstawie nazw podfolderów obecnych w folderze treningowym. Dodatkowo korzystamy z argumentów nazwowych:  
* `target_size` aby obrazy były pomniejszane do rozdzielczości `80 x 80`
* `batch_size` aby na raz z dysku wczytywana była pewna liczba obrazów - my wybierzemy w tym wypadku `25`
* `class_mode` za pomocą którego informujemy generator, że mamy do czynienia z zadaniem dychotomicznym (pies vs kot) do którego stosujemy jeden neuron wyjściowy. Informujemy o tym generator podając temu argumentowi wartość `'binary'`.

In [ ]:
train_generator = data_gen.flow_from_directory(
    r'________________',
    target_size=(__, __), ________=____, ________=____)

Generator powinien zakomunikować, że znalazł 22500 obrazów należących do 2 klas. Tekstowe etykiety klas są zgodne z nazwami podfolderów. Numeryczne etykiety klas również pochodzą od nazw folderów: konkretnie od posortowanej kolejności ich nazw. Możemy te informacje sprawdzić za pomocą atrybutu `class_indices` naszego generatora:

In [ ]:
________.________

Generator na raz będzie wczytywał `125` zdjęć z dysku. Przy czym generator pobierałby obrazy z folderu w nieskończoność, gdy dojdzie do ostatniego zdjęcia, wraca do pierwszego. Dlatego musimy sami pilnować ile "pobrań" z folderu stanowi przejście przez wszystkie zdjęcia - jeżeli chcemy aby w jedenej epoce treningowej sieć oglądała wszystkie zdjęcia treningowe. Sprawdzamy w związku z tym na ile "paczek" po 125 obrazów dzieli się nasz zbiór treningowy 22500 zdjęć:

In [ ]:
____ / ____

## Budujemy sieć

Do sieci typu CNN potrzebujemy dodatkowych warstw:

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

### Instrukcje
Zbuduj sieć postępując wedle poniższych zaleceń:
* co najmniej trzy warstwy splotowe (w sumie)
* max pooling co najmniej dwa razy
* max pooling za każdym razem po jednej lub dwóch warstwach splotowych
* pierwsza warstwa splotowa musi mieć co najmniej 8 filtrów
* nie używaj filtrów mniejszych niż `(3, 3)`
* każda kolejna warstwa splotowa powinna mieć większą ilość filtrów niż poprzednia
* jeżeli chcesz użyć więcej niż trzech warstw splotowych - częściej stosuj max pooling po dwóch warstwach splotowych niż jednej (max pooling najbardziej redukuje rozmiar obrazu, więc przy zbyt wielu można łatwo doprowadzić
* po zakończeniu części z warstwami splotowymi (i max poolingiem)
  dodaj warstwę `Flatten`, a następnie warstę `Dense` o dowolnej liczbie neuronów większej niż 25
* na sam koniec warstwa `Dense` z jednym neuronem (bo mamy tylko dwie kategorie, więc neuron wyjściowy będzie nam mówił "czypies" - niska aktywność znaczyć będzie kot) o aktywacji `'sigmoid'`

Im więcej warstw splotowych tym potencjalnie wyższa będzie poprawność Twojej sieci.

### Przykłady dodawania pojedynczych warstw:
* dodajemy warstwę splotową:
  ```python
  model.add(Conv2D(20, (5, 5), activation='relu'))
  ```
* dodajemy warstwę max pooling:
  ```python
  model.add(MaxPooling2D(pool_size=(2, 2)))
  ```
  (możecie zmienić `pool_size` na np `(3, 3)` aby uzyskać trzykrotną redukcję obrazu)
* dodajemy warstwę flatten:
  ```python
  model.add(Flatten())
  ```
* dodajemy warstwę Dense:
  ```python
  `model.add(Dense(32, activation='relu'))
   ```

In [ ]:
model = Sequential()

# schemat pierwszej warstwy jest już dodany, wystarczy go uzupełnić
model.add(Conv2D(__, (_, _), input_shape=(80, 80, 3), activation='relu'))

# kolejne warstwy...




# na koniec, flatten i dense


# i neuron wyjściowy, który mówi pies albo kot


In [ ]:
# i kompilujemy tak jak zwykle
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Sprawdź podsumowanie modelu:

In [ ]:
model.______()

*uwaga, poniższy komentarz dotyczy jednej sieci, którą zbuduję, wasza może być inna*  
Zauważ, że sieć ma pięć warstw splotowych oraz w sumie ponad 120 tysięcy parametrów (wag połączeń) - a na współczesne standardy zbudowaliśmy dosyć małą sieć.  
Zastanów się w której warstwie skupia się najwięcej parametrów sieci - znajdziesz to w tabeli powyżej. Dlaczego tak jest?

Do treningu korzystamy teraz nie z metody `.fit()`, ale `.fit_generator()` ponieważ podawać będziemy sieci jako pierwszy argument generator obrazów wczytujący je bezpośrednio z dysku.
Trening dużej sieci na komputerze bez wspomagania GPU będzie trwał długo, z tego względu ustawimy liczbę epok tylko na `10` (przy takiej liczbie epok będziemy mieli ograniczoną poprawność, ale w ramach projektu będziecie mieli szanse potrenować sieci splotowe dłużej). Dodatkowo używając zbyt wielu epokach możemy przetrenować sieć doprowadzając do świetnej poprawności na danych treningowych, ale słabej na danych testowych (ale takie ryzyko jest przy dużo większej liczbie epok niż 10). Ryzyko przetrenowania jest całkiem realne ponieważ nie stosujemy żadnej regularyzacji. O regularyzacji opowiemy sobie później.  
Argumentowi `steps_per_epoch` musisz podać wartość numeryczną, którą sprawdzaliśmy jeszcze przed treningiem sieć - pomyśl jaką.

In [ ]:
history = model.fit_generator(________, epochs=____, steps_per_epoch=____)

Zauważ że już po drugim przejściu przez dane (epoce) poprawność wynosi ok. 75% i stosunkowo szybko rośnie w kolejnych przejściach. To może niestety być zbyt piękne aby było prawdziwe - sugeruje że (przynajmniej częściowo) sieć zapamiętuje przykłady zamiast wyuczyć się wzorców pomagających rozpoznać psy i koty. Sprawdźmy to - zobaczmy jaką mamy poprawność na danych treningowych i testowych:

In [ ]:
plt.plot(history.history['acc'])
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Correctness', fontsize=12)
plt.grid(color=(0.85, 0.85, 0.85))

Moglibyśmy sieć trenować jeszcze dłużej, ale przydałby się nam do tego stacjonarny komputer z porządną kartą graficzną.
Sprawdzimy teraz poprawność na danych treningowych już wytrenowanej sieci (uwaga, może to chwilę potrwać!):

In [ ]:
loss, acc = model.evaluate_generator(____________, steps=180)
print('Poprawność na danych treningowych: {:.3f}%'.format(acc * 100))

Poprawność powinna wynosić coś między 70% a 90% (to zależy od złożoności trenowanej sieci - ja przy pierwszym treningu miałem 92%, a przy drugim 88%, ale kożystałem z 6 warstw splotowych). Ciekawi nas teraz czy podobnie wysoką poprawność otrzymamy na danych testowych.

Aby sprawdzić poprawność na danych testowych musisz utworzyć generator obrazów dla folderu `validate` - tak samo jak robiliśmy to dla folderu `train`:

In [ ]:
data_gen_val = ________
validate_generator = ___________

In [ ]:
data_gen_val = ImageDataGenerator(rescale=______)

validate_generator = data_gen_val.flow_from_directory(
    r'_____________________________________',
    target_size=(80, 80), batch_size=125,
    class_mode='binary')

In [ ]:
loss, acc = model.evaluate_generator(_______________, steps=180)
print('Poprawność na danych testowych: {:.3f}%'.format(acc * 100))

Poprawność na danych testowych jest znacząco wyższa od 75% (u mnie: 86% za pierwszym razem, 85% za drugim), więc model nauczył się czegoś sensownego, ale jest też niższa od poprawności na danych treningowych (u mnie o 6%), a więc możemy mieć do czynienia z przetrenowaniem (*overfitting*). Spróbujemy temu zapobiec później dodając do sieci regularyzację oraz image augmentation.  
Zanim jednak przejdziemy do tego kroku zrobimy analizę predykcji sieci.
Zaczniemy od wczytania kilku obrazów z danych walidacyjnych i sprawdzenia predykcji sieci:

In [ ]:
from emosie import load_images

Wczytamy najpierw 25 obrazów kotów, jako pierwszy argument podajemy folder treningowy zawierający koty, argumenty nazwowe powinny być jasne:

In [ ]:
cat, y_cat = load_images(r'________',
                         n_images=____, rescale=____, target_size=(___, ___))

Zobacz sobie kształt macierzy `cat`:

In [ ]:
cat._____

Teraz wczytamy tyle samo obrazów psów (teraz podajemy folder trenigowy dla psów):

In [ ]:
dog, y_dog = load_images(r'________',
                         n_images=____, rescale=____, target_size=(___, ___))

Porównaj wartości `y` dla psów i kotów, z czymś powinny Ci się one zgadzać:

In [ ]:
y_cat[0], y_dog[1]

Wreszcie wyświetlmy któryś z obrazów z macierzy `dog`:

In [ ]:
plt.imshow(dog[___])

oraz predykcję modelu dla tego obrazu (dodatkowy nawias kwadratowy wokół indeksownika prowadzi do zachowania wymiaru - **przykład prowadzącego**):

In [ ]:
model.predict(dog[[___]])

Co oznacza otrzymana wartość? Przypomnij sobie co mówiliśmy o predykcjach sieci gdy analizowaliśmy dane Fashion MNIST oraz zastanów się ile teraz mamy neuronów na wyjściu i co one nam mówią? Zastanów się co mogłaby oznaczać predykcja 0, a co predykcja 1, a wreszcie jak interpretować wartości pośrednie (między 0 a 1).

Zrób to samo dla któregoś obrazu kota:

Narysujmy teraz obraz z predykcją w tytule. Przeczytaj poniższy kod i uzupełnij `_____` w zmiennej tekstowej `msg`. Możesz zmieniać wartość `idx` aby zobaczyć różne obrazy. Aby zamiast psa zobaczyć jakiś obraz kota zmień w drugiej linijce `dog` na `cat`, a w trzeciej `y_dog` na `y_cat`.

In [ ]:
idx = 13
img = dog[[idx]]
true_label = y_dog[idx]

labels = ['kot', 'pies']
pred = model.________(img)[0, 0]

msg = 'Predykcja: {:.2f}% pewności, że to _____,\nnaprawdę to {}'
msg = msg.format(pred * 100, labels[true_label])

plt.imshow(img[0])
plt.axis('off')
plt.title(msg)

Połączymy teraz macierze `dog` oraz `cat` w jedną macierz `X_test` oraz wektory `y_dog` i `y_cat` w jeden wektor `y_test`

In [ ]:
X_test = np.concatenate([___, ___])
y_test = np.concatenate([___, ___])

Sprawdźmy ich kształt:

In [ ]:
print(X_test.____)
print(y_test.____)

Wygenerujmy teraz predykcję dla wszystkich obrazów:

In [ ]:
preds = model.predict(____)

i porównamy z poprawnymi odpowiedziami na wykresie poniżej:

In [ ]:
plt.plot(_____[:, 0], label='predicted')
plt.plot(_____, label='true')

plt.legend(loc='best')

Widzimy, że nawet w tych losowo wybranych 50 obrazach (25 kotów i 25 psów) sieć często nie jest pewna i dosyć często popełnia też błędy. Teraz wyświetlimy wszystkie wybrane 50 zdjęć posortowane względem wartości predykcji sieci. Na czerwono zobaczysz błędne predykcje:

In [ ]:
fig, ax = plt.subplots(nrows=10, ncols=5, figsize=(15, 30))

# znajdujemy kolejność sortującą od najwyższej do najniższej poprawności
idx_ord = np.argsort(preds[:, 0])[::-1]

colors = ['red', 'green']
for idx in range(len(idx_ord)):
    # wybieramy dany obraz
    img_idx = idx_ord[idx]
    img = X_test[img_idx]
    
    # znajdujemy predykcję dla niego
    this_pred = preds[img_idx, 0]
    categ = np.round(this_pred)

    # sprawdzamy czy predykcja jest poprawna
    corr = y_test[img_idx] == categ

    # przekształcamy indeks na adres kolumny i wiersza
    row = int(np.floor(idx / 5))
    col = int(idx - row * 5)
    
    # wyświetlamy obraz
    ax[row, col].imshow(img)
    ax[row, col].set_axis_off()
    
    # dodajemy tytuł
    msg = '{:.2f}%'.format(this_pred * 100)
    ax[row, col].set_title(msg, color=colors[int(corr)])

## Image augmentation
*image augmentation* to forma tworzenia częściowo nowych danych do treningu w oparciu o dane które już mamy. Opiera się na założeniu, że jeżeli zdjęcie przedstawia np. psa, to pewne operacje na tym obrazie jak np. przybliżenie, obrót, przesunięcie itp. cały czas nie zmieniają kategorii obiektu obecnego na zdjęciu. Innymi słowy: jeżeli obrócimy zdjęcie kota o 25 stopni i zrobimy zoom 10% to cały czas zdjęcie przedstawia kota. W ten sposób możemy zwiększyć ilość zdjęć treningowych dla sieci.
Zobacz przykład poniżej:

In [ ]:
check_gen = ImageDataGenerator(rescale=scaling, zoom_range=[0.8, 1.2], horizontal_flip=True,
                              rotation_range=45, width_shift_range=0.2, height_shift_range=0.2)
check_generator = check_gen.flow_from_directory(
    r'C:\Users\mmagn\Dropbox\DANE\ML\psy_vs_koty\check_aug',
    target_size=(80, 80), batch_size=1, class_mode='binary')

In [ ]:
fig, ax = plt.subplots(ncols=5, nrows=5, figsize=(12, 12))
ax = ax.ravel()

for idx, (X, y) in enumerate(check_generator):
    if idx >= 25:
        break
    ax[idx].imshow(X[0])
    ax[idx].set_axis_off()

Aby oszczędzić czas - zamiast tworzyć i trenować sieć od nowa, my będziemy kontynuować trening poprzedniej sieci, korzystając tylko z innego generatora obrazów.  
(gdy wywołujemy metodę `.fit()` modelu po raz drugi, to faktycznie kontynuujemy trening, jeżeli chcemy zrestetować model najłatwiej jest utworzyć go od nowa - tzn. np. odpalić jeszcze raz komórkę, w której definiujemy model)

In [ ]:
aug_gen = ImageDataGenerator(rescale=scaling, zoom_range=[0.8, 1.2], horizontal_flip=True,
                              rotation_range=45, width_shift_range=0.2, height_shift_range=0.2)
augmented_generator = aug_gen.flow_from_directory(
    r'__________________________________________',
    target_size=(80, 80), batch_size=125,
    class_mode='binary')

Uwaga, trening z modyfikacjami obrazu trwa jeszcze wolniej niż wcześniej ponieważ każdy obraz nie jest tylko wczytywany z dysku, ale jeszcze modyfikowany. Postępy w treningu będą też mniejsze z kolejnymi epokami, ponieważ zadanie jest trudniejsze: sieć nie może już polegać na sztywnym zapamiętywaniu pewnych własności zdjęć - musi nauczyć się teraz rozpoznawać cechy obrazów, które będą niezmienne względem przesunięć, obrotu, przybliżenia itp.

In [ ]:
history2 = model.fit_generator(augmented_generator, epochs=10, steps_per_epoch=180)

In [ ]:
plt.plot(history2.history['acc'])

Kiedy jednak testujemy sieć na danych testowych, nie stosujemy już modyfikacji obrazu. Modyfikacje to tylko zabieg mający wspomóc uczenie sieci.

In [ ]:
loss, acc = model.evaluate_generator(validate_generator, steps=180)
print('Poprawność na danych testowych to teraz: {:.3f}%'.format(acc * 100))

Poprawność powinna trochę podskoczyć. Aby mieć jednak dobre porównanie treningu z oraz bez modyfikacji obrazu, musielibyśmy: a) zresetować model i trenować go od początku z modyfikacjami obrazu oraz b) trenować go dłużej - ponieważ trening z modyfikacjami jest trudniejszy.  

Moglibyśmy na przykład trenować model z/bez modyfikacji za każdym razem tak długo, aż osiągnie poprawność 95% na danych treningowych. Do takiego trenowania warunkowego przydaje się obiekt `EarlyStopping` - ale o tym powiemy sobie następnym razem.

## Regularyzacja sieci
Budujemy całą sieć tak samo jak wcześniej, dodajemy jednak `Dropout` do jednej z ostatnich warstw.
Dropout to prosty sposób regularyzacji - z danej warstwy losowo co pewien czas usuwana jest część neuronów. Prowadzi to do dużo wolniejszego uczenia się, ale zapobiega sytuacji w której neurony bądź ich kombinacje zapamiętują konkretne przykłady treningowe. 
`Dropout(0.5)` prowadzi do wyrzucenia 50% neuronów przy każdym etapie treningowym. W naszym wypadku chcemy odrzucać właśnie 50% neuronów przy treningu.

### Ćwiczenie
Przeklej poniżej oryginalną definicję sieci. Dodaj dropout po warstwie `Flatten`. Pamiętaj że dodajemy element do sieci za pomocą `model.add()`.

Regularyzacja prowadzi do dużo wolniejszego uczenia się, dlatego tym razem będziemy trenować przez więcej epok - co najmniej 20.
To trochę potrwa, także cierpliwości...

In [ ]:
# tutaj trenuj model

In [ ]:
# sprawdź jego poprawność

Inną strategią jest tzw. *transfer learning* czyli skorzystanie z sieci wcześniej wytrenowanej na milionach zdjęć (jest sporo takich dostępnych publicznie - wliczając kilka bardzo znanych architektur takich jak np. `'Inception'` czy `'ResNet'`) i dotrenowanie jej do naszego zastosowania. Robimy to zwykle podmieniając w takiej wytrenowanej sieci kilka ostatnich pięter na np. kilkaset neuronów `Dense` pozwalających na klasyfikację pies vs kot. Wtedy trening odbywałby się na takiej hybrydowej sieci - duża część już zastygła, nietrenowana, bo wytrenowana wcześniej w wyłapywaniu z obrazu cech wartościowych dla klasyfikacji różnych obiektów, a tylko końcowa warstwa (albo warstwy) są trenowane z wykorzystaniem posiadanych danych.  
Nie przerobimy już jednak tego podejścia na zajęciach.